In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# 1.c

In [ ]:
cutoff_frequency = 1e3 * 1e-9  # Ohm * Farad = Hz
low_pass = lambda x: 1 / (1 + (x * cutoff_frequency) ** 2)
high_pass = lambda x: 1 / (1 + 1 / (x * cutoff_frequency) ** 2)
frequencies = np.logspace(3, 9, 1000)

plt.loglog(frequencies, low_pass(frequencies), label="low-pass")
plt.loglog(frequencies, high_pass(frequencies), label="high-pass")
plt.ylabel("Gain")
plt.xlabel("Frequency (Hz)")
plt.legend()

pass

# 2

In [ ]:
ms = 1e-3


def plot_y():
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    ax.grid(True, c="0.8")
    ax.set_xlabel("$t$ [ms]")
    ax.set_ylabel("$y(t)$")
    return ax

# 2.c

In [ ]:
y = lambda t: 1 - np.exp(-t/(10*ms))

time = np.linspace(0, 100 * ms, 500)
ax = plot_y()

ax.plot(time / ms, y(time))
print("no overshooting")

# 2.f

In [ ]:
def y(t, 𝜁):
    𝜔0 = 2 * np.pi / ms
    if 𝜁 < 1:
        a = np.sqrt(1 - 𝜁**2)
        return 1 - np.exp(-𝜁 * 𝜔0 * t) / a * np.sin(𝜔0 * a * t + np.arctan(a / 𝜁))
    elif 𝜁 == 1:
        return 1 - np.exp(-𝜔0 * t) - t * 𝜔0 * np.exp(-𝜔0 * t)
    elif 𝜁 > 1:
        b = np.exp(-𝜁 * 𝜔0 * t)
        c = np.sqrt(𝜁**2 - 1)
        d = 𝜔0 * c * t
        return 1 - b * np.cosh(d) - 𝜁 / c * b * np.sinh(d)


time = np.linspace(0, 3 * ms, 500)
ax = plot_y()

for 𝜁 in (1 / 2, 1, 3 / 2):
    ax.plot(time / ms, y(time, 𝜁), label=𝜁)

ax.legend(title="ζ")
pass

# 2.g

In [ ]:
time = np.linspace(0, 2 * ms, 500)
ax = plot_y()
for 𝜁 in (0.01, 0.1, 0.2, 0.4, 0.6):
    ax.plot(time / ms, y(time, 𝜁), label=𝜁)

ax.legend(title="ζ")
print("overshoot if ζ<1")
print("larger overshoot for smaller ζ")

# 3.

In [ ]:
tau = np.pi * 2


def unwrap(x):
    try:
        return np.unwrap(x)
    except:
        return x


magnitude_calc = lambda f, s: 20 * np.log10(np.abs(f(s)))
phase_calc = lambda f, s: 180 / np.pi * unwrap(np.angle(f(s)))


def bode_plot(func, f_min, f_max, n_points=1000):
    omega_scalar = np.logspace(np.log10(f_min), np.log10(f_max), n_points)
    omega = tau * omega_scalar
    magnitude = magnitude_calc(func, 1j * omega)
    phase = phase_calc(func, 1j * omega)
    fig, (ax1, ax2) = plt.subplots(
        2, 1, sharex=True, constrained_layout=True, figsize=(8, 5)
    )

    ax1.semilogx(omega_scalar, magnitude, "k")
    ax1.grid(True, c="0.8")
    ax1.set_ylabel("Magnitude (dB)")

    ax2.semilogx(omega_scalar, phase, "k")
    ax2.grid(True, c="0.8")
    ax2.set_xlabel("Frequency $f$ (Hz)")
    ax2.set_ylabel("Phase (deg)")
    tick_range = (math.floor(min(phase) / 45) * 45, math.ceil(max(phase) / 45) * 45)
    ax2.set_yticks(np.arange(tick_range[0], tick_range[1], 45))

    plt.xlim(omega_scalar[0], omega_scalar[-1])
    return (ax1, ax2)

# 3.a

$𝑇(𝑠) = 10^{12}(2\pi × 1 \text{Hz})^2(𝑠 + 2\pi × 1 \text{Hz}) \left/ \left((𝑠 + 2\pi × 1 \text{kHz}) (𝑠 + 2\pi × 1 \text{MHz})^2\right)\right.$

In [ ]:
T = lambda s: 1e12 * tau**2 * (s + tau * 1) / ((s + tau * 1e3) * (s + tau * 1e6) ** 2)

bode_plot(T, 1e-3, 1e9)
pass

# 3.c

In [ ]:
axs = bode_plot(T, 1e-3, 1e9)
zeros_f = np.array([1e0])
poles_f = np.array([1e3, 1e6])
for ax in axs:
    for f in zeros_f:
        ax.axvline(x=f, c="b", ls="--")
    for f in poles_f:
        ax.axvline(x=f, c="r", ls="--")

corner_f = np.concatenate((zeros_f, poles_f))
omega = tau * corner_f
phase = phase_calc(T, 1j * omega)
print("phase at corner f's: ", phase)
print("phase/45° at corner f's: ", phase / 45)
print("⇒ all are ≈//45°")

$𝑇_2(𝑠) = (2\pi × 1 \text{kHz}) \left/ \left((𝑠 + 2\pi × 1 \text{kHz})\right)\right. $

In [ ]:
T2 = lambda s: (tau**2 * 1e3) / (s + tau * 1e3)  # Hz

axs = bode_plot(T2, 1e-1, 1e7)
for ax in axs:
    ax.axvline(x=1e3, c="r", ls="--")

# 3.f

In [ ]:
print("magnitude: {}".format(magnitude_calc(T, 0)))
print("phase: {}".format(phase_calc(T, 0)))

In [ ]:
print("magnitude: {}".format(magnitude_calc(T2, 0)))
print("phase: {}".format(phase_calc(T2, 0)))